In [1]:
import json
import math
import pickle
import random
import time
import itertools
from re import S
from mip import *
import numpy as np
import pandas as pd
import numpy.random as rnd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et
from collections import defaultdict

In [2]:
def iter_docs(author):
    author_attr = author.attrib
    for doc in author.iter('vehicle'):
        doc_dict = author_attr.copy()
        doc_dict.update(doc.attrib)
        #doc_dict['data'] = doc.text
        yield doc_dict

In [3]:
def iter_author(etree):
    for author in etree.iter('timestep'):
        for row in iter_docs(author):
            yield row

In [4]:
def parse_sumo_traces(ts):
    data = et.parse('here.output9.xml')
    df1 = pd.DataFrame(list(iter_author(data)))
    df = df1[['time','id','lane']].copy()
    df[['start','end']] = df['lane'].str.split('to',1, expand=True)
    df['start'] = df['start'].str.split('_').str[0]
    df['start'] = df['start'].str.replace(r":n","")
    df['end'] = df['end'].str.split('_').str[0]
    df['end'] = df['end'].fillna(value=df['start'])
    df['time'] = df['time'].astype('float')
    df['time'] = df['time']/(3600/ts)
    df['time'] = df['time'].astype('int')
    df['start'] = df['start'].astype('int')
    df['end'] = df['end'].astype('int')
    df = df.drop(columns=['lane'])
    return df

In [5]:
def nextfit(cpu_weights, gpu_weights, c_cpu, c_gpu):
    """ Next fit heuristic for demand estimation.
    Find out how many fog nodes are needed with given capacities and weights.
    """
    if sum(cpu_weights) + sum(gpu_weights) == 0:
        res = 0
    else:
        res = 1
        rem_cpu = c_cpu
        rem_gpu = c_gpu
        for _ in range(len(cpu_weights)):
            if rem_cpu >= cpu_weights[_] and rem_gpu >= gpu_weights[_]:
                rem_cpu -= cpu_weights[_]
                rem_gpu -= gpu_weights[_]
            else:
                res += 1
                rem_cpu = c_cpu - cpu_weights[_]
                rem_gpu = c_gpu - gpu_weights[_]
    return res

In [6]:
class capacity_planner():
    def __init__(self):
        # simulation settings
        self.r =  20        # number of regions
        self.clusters = pd.read_pickle('clusters_%i.pkl'%self.r)   # division of regions
        self.n_cl = len(self.clusters)   # equals r
        self.ts = 12
        self.E1 = 24         # length of the episode in hour
        self.E2 = 12        # lenth of the hour in 5 minutes
        self.n_vfn = 30     # number of VFNs
        self.c_tra = 1      # traveling cost per minute
        self.c_rent = 5    # rental cost per VFN per hour
        self.c_ser = 0.01   # hardware cost per 5 minutes
#         self.d_adjust = [[0 for i in range(self.r)] for j in range(12)]     # adjusted demand matrix with different QoS levels
#         self.w_adjust = [0 for i in range(self.r)]         # adjusted demand in each hour at the current hour
        
        # centroids of each region
        center = pd.read_pickle('cen_list_%i.pkl'%self.r)
        self.cens = []
        for cl in self.clusters:
            for i, seg in enumerate(cl):
                if seg in center:
                    self.cens.append(i)

        # Distance matrix from RG map
        with open('D_list_%i.pkl'%self.r,'rb') as f:
            self.DD = pickle.load(f)

        # Cellular connection matrix from map overlay
        with open('C_list_%i.pkl'%self.r,'rb') as f:
            self.HH = pickle.load(f)
        if self.r == 20:
            self.HH[10][3][3]=1

        # Distance matrix from CS map
        with open('D_cl_%i.pkl'%self.r,'rb') as f:
            self.DCL = pickle.load(f)
    
    
    # gernerate user profiles
    def generate_user_profiles(self, nv, df):
        """Generate user profiles:
        - `counts`: how many users,
        - `ids`: user ids,
        - `locs`: road segment id in the cluster for the user,
        - `cpu`:
        - `gpu`:
        - `price`:
        for each timestamp and each cluster/region"""
#         ser = []
#         lat = []
#         for i in range(nv):
#             ser.append(random.randint(0,3))
#             lat.append(random.randint(0,2))
        with open('ser_list.pkl','rb') as f:
            ser_list = pickle.load(f)
        with open('lat_list.pkl','rb') as f:
            lat_list = pickle.load(f)
        ser = ser_list[self.time]
        lat = lat_list[self.time]

        # Application profile and service level agreement
        # the prices can be tuned
        app = pd.read_csv('app_profile.csv', delimiter=';', decimal=",")
        app = app.set_index('tp')
        cpu = []
        gpu = []
        price = []
        for i in range(nv):
            element = ser[i]
            value = lat[i]
            cpu.append(app.values.tolist()[element][value+3])
            gpu.append(app.values.tolist()[element][value+6])
            price.append(app.values.tolist()[element][value+9])
        n_cl = len(self.clusters)
        cl_st = []
        loc_st = []
        for i,row in df.iterrows():
            for cl, seg in enumerate(self.clusters):
                if row['start'] in seg:
                    cl_st.append(cl)
                    loc_st.append(seg.index(row['start']))
        df['cl'] = cl_st
        df['loc'] = loc_st
        counts = []
        ids = []
        locs = []
        for i in range(self.ts):
            time = df[df['time']==i]
            count = np.zeros(len(self.clusters))
            id = [ [] for _ in range(len(self.clusters)) ]
            loc = [ [] for _ in range(len(self.clusters)) ]
            for _,t in time.iterrows():
                for j in range(len(self.clusters)):
                    if t['cl'] == j:
                        count[j] += 1
                        id[j].append(t['id'])
                        loc[j].append(t['loc'])
            counts.append(count)
            ids.append(id)
            locs.append(loc)
        return counts, ids, locs, cpu, gpu, price, ser, lat
    
    
    # task generation
    def task_generation(self, n_pre, level, ids, cpu, gpu, price):
        """
        Parametes:
        - `level`: Level of Quality of Service (QoS), 0 = highest (all accepted), 11 = lowest (only the most expensive ones)
        Returns:
        - `nums`: The maximum number of fog nodes needed in all short timestamps (st) for each cluster
        - `revenues`: Total revenues for all the st for each cluster
        - `cpu_ws`: CPU demand for cluster*st
        - `gpu_ws`: GPU demand for cluster*st
        - `prices`: Prices for cluster*st
        """
        # nums = []
        revenues = []
        cpu_ws = []
        gpu_ws = []
        prices = []
        for row in ids:
            for cluster, users in enumerate(row):
                cpu_weights = []
                gpu_weights = []
                pris = []
                for index, user in enumerate(users):
                    if index+1 <= n_pre[cluster] * (1 - 0.1 * level):
                        cpu_weights.append(cpu[user]) 
                        gpu_weights.append(gpu[user]) 
                        pris.append(price[user])
                    else:
                        cpu_weights.append(0.0) 
                        gpu_weights.append(0.0) 
                        pris.append(0.0)
                # c_cpu = 1
                # c_gpu = 1
                # result = nextfit(cpu_weights, gpu_weights, c_cpu, c_gpu)
                # nums.append(result)
                revenues.append(int(sum(pris)))
                cpu_ws.append(cpu_weights)
                gpu_ws.append(gpu_weights)
                prices.append(pris)
        # nums = np.array(nums).reshape(self.ts,self.n_cl).max(axis=0).tolist()
        revenues = np.array(revenues).reshape(self.ts,self.n_cl).sum(axis=0).tolist()
        prices = np.array(prices).reshape(self.ts,self.n_cl).T
        cpu_ws = np.array(cpu_ws).reshape(self.ts,self.n_cl).T
        gpu_ws = np.array(gpu_ws).reshape(self.ts,self.n_cl).T
        return revenues, cpu_ws, gpu_ws, prices
    
    
    # demand estimation
    def demand_estimation(self, n_pre, level):
        """
        Parametes:
        - `level`: Level of Quality of Service (QoS), 0 = highest (all accepted), 11 = lowest (only the most expensive ones)
        Returns:
        - `nums`: The maximum number of fog nodes needed in all short timestamps (st) for each cluster
        - `revenues`: Total revenues for all the st for each cluster
        - `cpu_ws`: CPU demand for cluster*st
        - `gpu_ws`: GPU demand for cluster*st
        - `prices`: Prices for cluster*st
        """
        nums = []
        revenues_pre = []
        app = pd.read_csv('app_profile.csv', delimiter=';', decimal=",")
        app = app.set_index('tp')
        # cpu_ws = []
        # gpu_ws = []
        # prices = []
        for row in range(self.ts):
            for cluster in range(self.r):
                cpu_weights = []
                gpu_weights = []
                pris = []
                for index in range(n_pre[cluster]):
                    if index+1 <= n_pre[cluster] * (1 - 0.1 * level):
                        element = random.randint(0,3)
                        value = random.randint(0,2)
                        cpu_weights.append(app.values.tolist()[element][value+3])
                        gpu_weights.append(app.values.tolist()[element][value+6])
                        pris.append(app.values.tolist()[element][value+9])
                    else:
                        cpu_weights.append(0.0) 
                        gpu_weights.append(0.0) 
                        pris.append(0.0)
                c_cpu = 1
                c_gpu = 1
                result = nextfit(cpu_weights, gpu_weights, c_cpu, c_gpu)
                nums.append(result)
                revenues_pre.append(int(sum(pris)))
                # cpu_ws.append(cpu_weights)
                # gpu_ws.append(gpu_weights)
                # prices.append(pris)
        nums = np.round(np.array(nums).reshape(self.ts,self.n_cl).mean(axis=0),0).tolist()
        revenues_pre = np.array(revenues_pre).reshape(self.ts,self.n_cl).sum(axis=0).tolist()
        # prices = np.array(prices).reshape(self.ts,self.n_cl).T
        # cpu_ws = np.array(cpu_ws).reshape(self.ts,self.n_cl).T
        # gpu_ws = np.array(gpu_ws).reshape(self.ts,self.n_cl).T
        return nums, revenues_pre
    
    
    ## city scale capacity planning
    # Problem formulation of CS routing
    def CS_VFN_routing(self, D, O, demand, rev, t):
        """ Problem formulation of city-scale (CS) routing
        Returns:
        - `model_cs.objective_value`: optimal profit of CS planning
        - `location_cs`: which cluster the vehicle is located. 0 = depot
        - `distribution_cs`: how many vehiles are in each cluster and depot (0)
        - `package_cs`: QoS level chosen for each cluster
        """
        start_time = time.time()
        v = self.n_vfn
        r = self.r
        V = range(v)         # set of VFNs
        R = range(r+1)       # set of locations, 0 is the depot
        T = range(t)
        model_cs = Model(solver_name=CBC)

        # Decision variable
        x = [[model_cs.add_var(var_type=BINARY) for j in R] for i in V]   # x_ij is a binary variable to represent whether fog node i will go to region j
        y = [[model_cs.add_var(var_type=BINARY) for k in T] for j in R]   # y_jk is a binary variable to represent whether the region j will provide service package k

        # Constraints
        for j in R:
            model_cs += xsum(x[i][j] for i in V) >= xsum(demand[j][k]*y[j][k] for k in T)     # demand fulfilment connstraint
            model_cs += xsum(y[j][k] for k in T) == 1       # one service package constraint
        for i in V:
            model_cs += xsum(x[i][j] for j in R) == 1       # one-destination constraint

        # Objective function
        opt_cs = xsum(rev[j][k]*y[j][k] for j in R for k in T) - self.c_tra*(xsum(D[i][j]*x[i][j] for i in V for j in R)) - self.c_rent*(xsum(O[i][j]*x[i][j] for i in V for j in R))
        model_cs.objective = maximize(opt_cs)
        model_cs.optimize(max_seconds= 300)
        time_cs = time.time() - start_time

        # CS operational cost and CS location of the VFNs
        action_cs = []
        for i,var in enumerate(model_cs.vars):
            action_cs.append(var.x)
        action1 = np.reshape(action_cs[:v*(r+1)], (v, r+1))
        action2 = np.reshape(action_cs[v*(r+1):], (r+1, t))
        # print(action2)
        location_cs = []
        for p in V:
            location_cs.append([i for i, element in enumerate(action1[p]) if element!=0])
        location_cs = sum(location_cs, [])
        # print('CS location:', location_cs)
        distribution_cs = list(np.zeros(r+1,))
        for q in R:
            for p, element in enumerate(V):
                if location_cs[p] == q:
                    distribution_cs[q] += 1
        # print('CS distribution:', distribution_cs)
#         package_cs = []
#         for o in R:
#             package_cs.append([i for i, element in enumerate(action2[o]) if element!=0])
#         package_cs = sum(package_cs, [])
#         package_cs = package_cs[1:]
        package_cs = [0]*r
        # print('Package selection:', package_cs)
        # print(model_cs.objective_value)
        return model_cs.objective_value, location_cs, distribution_cs, package_cs, time_cs
    
    
    # Calculate the CS operational costs
    def calculate_cs_opt_cost(self, d_list, revenue_list, D, O, t):
        """Pre-processing for CS_VFN_routing
        - add depot
        """
        d_list1 = [np.insert(d, 0, 0) for d in d_list]
        demand = np.array(d_list1).T
        revenue_list1 = [np.insert(revenue, 0, 0) for revenue in revenue_list]
        rev = np.array(revenue_list1).T
        opt_cs, location_cs, distribution_cs, package_cs, time_cs = self.CS_VFN_routing(D, O, demand, rev, t)
        return opt_cs, location_cs, distribution_cs, package_cs, time_cs
    
    
    # CS routing simulation
    def CS_simulation(self):
        # operational matrix
        O = [list(np.insert(np.ones(self.r), 0, 0)) for i in range(self.n_vfn)]    # operational matrix: the VFN is operating when it is not at depot
        location_cs = np.zeros((self.n_vfn,), dtype=int)                       # initial location of the VFNs: all at the depot 0
        df = parse_sumo_traces(self.ts)
        pre = pd.read_pickle('flow_%i.pkl'%self.r).T
        self.distributions_cs = []
        self.opts_cs = []
        self.opts_cp = []
        self.packages = []
        self.times_cs = []
        self.revenues = []
        self.cpus = []
        self.gpus = []
        self.prices = []
        self.counts = []
        self.locs = []
        self.ser_list = []
        self.lat_list = []
        self.profit_list = []
        self.service_rate_list = []
        self.fulfill_rate_list = []
        self.opts_rg_block = []
        self.unserves_block = []
        self.idles_block = []
        self.rates_block = []
        self.tras_block = []
        self.times_rg_block = []
        for self.time in range(self.E1):
            print('Long Time step %d:' %self.time)

            # update distance matrix
            D = [[self.DCL[location_cs[i]][j] for j in range(self.r+1)] for i in range(self.n_vfn)]

            # parse sumo traces
            df_current = df[df['time'].isin(range(self.time*self.ts+1,(self.time+1)*self.ts+1))]
            df_current = df_current.reset_index(drop=True)
            df_current['time'] = df_current['time'] - self.ts*self.time
            lst = df_current.id.unique().tolist()
            nv = len(lst)
            dic = {lst[i]:i for i in range(0, len(lst) ) }
            df_current['id'] = df_current['id'].replace(dic)
            df_current['id'] = df_current['id'].astype('float')
            df_current['id'] = df_current['id'].astype('int')
            # print(nv)
            # print(df_current)

            # Generate the user profile
            count, ids, loc, cpu, gpu, price, ser, lat = self.generate_user_profiles(nv, df_current)
            self.ser_list.append(ser)
            self.lat_list.append(lat)
            self.counts.append(count)
            self.locs.append(loc)
            
            d_list = []
            revenue_list = []
            revenue_pre_list = []
            cpu_list = []
            gpu_list = []
            price_list = []
            
            for level in range(10):
                n_pre = pre[self.time]
                revs, cpu_ws, gpu_ws, prices_s = self.task_generation(n_pre, level, ids, cpu, gpu, price)
                d, revs_pre = self.demand_estimation(n_pre, level)
                d_list.append(d)
                revenue_list.append(revs)
                revenue_pre_list.append(revs_pre)
                cpu_list.append(cpu_ws)
                gpu_list.append(gpu_ws)
                price_list.append(prices_s)

            # Calculate CS operational cost
            opt_cs, locations_cs, distribution_cs, package_cs, time_cs = self.calculate_cs_opt_cost(d_list, revenue_pre_list, D, O, t=10)
            # opt_cp, locations_cp, distribution_cp, package_cp, time_cp = self.calculate_cs_opt_cost(d_list, revenue_list, D, O, t=1)
            self.distributions_cs.append(distribution_cs)
            opt_cs = opt_cs - sum([revenue_pre_list[element][i] for i,element in enumerate(package_cs)]) + sum([revenue_list[element][i] for i,element in enumerate(package_cs)])
            self.opts_cs.append(opt_cs)
            # self.opts_cp.append(opt_cp)
            self.packages.append(package_cs)
            # print('package:', package_cs)
            print('opts_cs:', opt_cs)
            # print('opts_cp:', opt_cp)
            self.times_cs.append(time_cs)
            
            self.revenues.append([revenue_list[element][i] for i,element in enumerate(package_cs)])
            self.cpus.append([cpu_list[element][i] for i,element in enumerate(package_cs)])
            self.gpus.append([gpu_list[element][i] for i,element in enumerate(package_cs)])
            self.prices.append([price_list[element][i] for i,element in enumerate(package_cs)])
            
            # regional planning
            profit, service_rate, fulfill_rate, opts_rg_list, unserves_list, idles_list, rates_list, rates1_list, tras_list, times_rg_list = self.capacity_plan_and_feeback()
            self.profit_list.append(profit)
            self.service_rate_list.append(service_rate)
            self.fulfill_rate_list.append(fulfill_rate)
            self.opts_rg_block.append(opts_rg_list)
            self.unserves_block.append(unserves_list)
            self.idles_block.append(idles_list)
            self.rates_block.append(rates_list)
            self.tras_block.append(tras_list)
            self.times_rg_block.append(times_rg_list)
        # return opts_cs, distributions_cs, packages, revenues, cpus, gpus, prices, opts_cp, locs, counts
        
    
    ## regional capacity planning
    # Problem formulation of RG routing
    def RG_VFN_routing(self, cpu_rg, gpu_rg,c_pen, w,u,s, D, H):
        start_time = time.time()
        W = range(w)
        S = range(s*2)
        U = range(u)
        # print('w, s, u:', w, s, u)
        # ser = [list(np.insert(np.ones(s), np.zeros(s), 0)) for l in W]
        ser = [np.ones(s).tolist()+np.zeros(s).tolist()]* w
        c_ser = 0.01
        model_rg = Model(solver_name=CBC)

        # Decision variables
        x = [[model_rg.add_var(var_type=BINARY) for m in S] for l in W]
        y = [[model_rg.add_var(var_type=BINARY) for n in U] for m in S]
        pen = [model_rg.add_var(var_type=BINARY) for n in U]

        # Constraints
        for l in W:
            model_rg += xsum(x[l][m] for m in S) == 1                   # one-destination constraint
        for m in S:
            model_rg += xsum(cpu_rg[n]*y[m][n] for n in U) <= xsum(x[l][m] for l in W)   # CPU constraint
            model_rg += xsum(gpu_rg[n]*y[m][n] for n in U) <= xsum(x[l][m] for l in W)   # GPU constraint
        for n in U:
            model_rg += pen[n] == 1 - xsum(H[m][n]*y[m][n] for m in S)  # service provision constraint

        # Objective function
        opt_rg = self.c_tra*(xsum(D[l][m]*x[l][m] for l in W for m in S)) +  xsum(c_pen[n]*pen[n] for n in U)+ c_ser*(xsum(ser[l][m]*x[l][m] for l in W for m in S))  # regional traveling and penalty costs
        model_rg.objective = minimize(opt_rg)
        model_rg.optimize(max_seconds= 300)
        time_rg = time.time() - start_time

        # RG operational cost, RG locations of VFNs, and task allocation
        action_rg = []
        for i,var in enumerate(model_rg.vars):
            action_rg.append(var.x)
        action1 = np.reshape(action_rg[:w*(2*s)], (w, (2*s)))
        action2 = np.reshape(action_rg[w*(2*s):w*(2*s)+(2*s)*u], ((2*s), u))
        action3 = action_rg[w*(2*s)+(2*s)*u:]
        location_rg = []
        service_rg = []
        unserved = []
        on = []
        idle = []
        tra_rg = float(self.c_tra*(xsum(D[l][m]*action1[l][m] for l in W for m in S)))
        # print(tra_rg)
        for l in W:
            location_rg.append([i for i, element in enumerate(action1[l]) if element!=0])
        location_rg = sum(location_rg, [])
        for loc in location_rg:
            if loc <= s:
                on.append(1)
            else:
                on.append(0)
        idle = len(on)-sum(on)
        # print('Idle VFN:', idle)
        for m in S:
            service_rg.append([i for i, element in enumerate(action2[m]) if element!=0])
        service_rg = [x for x in service_rg if x != []]
        unserved.append([i for i, element in enumerate(action3) if element!=0])
        unserved = sum(unserved, [])
        real_u = 0
        for num in U:
            if c_pen[num] != 0:
                real_u += 1
        serve = real_u - len(unserved)
        if real_u != 0:
            rate = serve/u
            rate1 = serve/real_u  
        else:
            rate = np.nan
            rate1 = np.nan
        return model_rg.objective_value, location_rg, service_rg, unserved, idle, rate, rate1, tra_rg, time_rg
    

    # Calculate the RG traveling costs
    def calculate_rg_opt_cost(self, cpu_rg, gpu_rg,c_pen, w,u,s, D, H):
        opt_rg, location_rg, service_rg, unserved, idle, rate, rate1, tra_rg, time_rg = self.RG_VFN_routing(cpu_rg, gpu_rg,c_pen, w,u,s, D, H)
        return opt_rg, unserved, idle, rate, rate1, tra_rg, time_rg
    
    
    # RG routing simulation
    def RG_simulation(self, cluster):
        D_rg = self.DD[cluster]
        H_rg = self.HH[cluster]
        w = int(self.distributions_cs[self.time][cluster+1])     # number of VFNs
        s = len(D_rg)          # number of cells
        E2 = 12         # length of the episode: 1 hour (12*5 min)
        user_locs = list(map(list, zip(*self.locs[self.time])))[cluster]
        user_nums = list(map(list, zip(*self.counts[self.time])))[cluster]
        location_rg = np.ones((w,), dtype=int)*self.cens[cluster]      # initial location of the VFNs: the centroids
        opts_rg =[]
        unserves = []
        idles = []
        tras = []
        rates = []
        rates1 = []
        times_rg = []
        for t in range(E2):
            print('Short Time step %d:' %t)
            u = len(self.cpus[self.time][cluster][t])     # number of users
            # print('user number:', u)

            # user cpu and gpu usage
            cpu_rg = self.cpus[self.time][cluster][t]
            gpu_rg = self.gpus[self.time][cluster][t]
            c_pen = [1.5 * pr for pr in self.prices[self.time][cluster][t]]
            # print('len_pen:',len(c_pen))

        #     # taxis price: 6:00 â€“ weekday 18:00 - 3.9/0.99, weekend / everyday 18:00 â€“ 6:00 6.9/1.09
        #     if weekday == True and 6 <= time < 18:
        #         c_tra = 0.99
        #     else:
        #         c_tra = 1.09
            user_loc = user_locs[t]
            # print(user_loc)

            # update distance and cellular connection matrix
        #     D = [[float(math.sqrt(
        #             ((coord_rg[location_rg[l]][0]-coord_rg[m][0])**2) +
        #             ((coord_rg[location_rg[l]][1]-coord_rg[m][1])**2) ))
        #             for m in range(s)]
        #             for l in range(w)]
            D = [[D_rg[location_rg[l]][m] for m in range(s)] for l in range(w)]
            D = [item+item for item in D]
            # print(len(D),len(D[0]))
            H = [[H_rg[m][user_loc[n]] for n in range(u)] for m in range(s)]
            B = np.zeros_like(H).tolist()
            H = [*H, * B]
            # print(len(H),len(H[0]))

            # calculate RG operational cost
            opt_rg, unserved, idle, rate, rate1, tra_rg, time_rg = self.calculate_rg_opt_cost(cpu_rg, gpu_rg, c_pen, w, u, s, D, H)
            print('RG cost:', opt_rg)
            opts_rg.append(opt_rg)
            unserves.append(unserved)
            idles.append(idle)
            rates.append(rate)
            rates1.append(rate1)
            tras.append(tra_rg)
            times_rg.append(time_rg)
        return opts_rg, unserves, idles, rates, rates1, tras, times_rg
    
    
    # Capacity plan and feeback
    def capacity_plan_and_feeback(self):
        opts_rg_list = []
        unserves_list = []
        idles_list = []
        rates_list = []
        rates1_list = []
        tras_list = []
        times_rg_list = []
        for i in range(self.r):
            print('cluster', i)
            opts_rg, unserves, idles, rates, rates1, tras, times_rg = self.RG_simulation(i)
            opts_rg_list.append(opts_rg)
            unserves_list.append(unserves)
            idles_list.append(idles)
            rates_list.append(rates)
            rates1_list.append(rates1)
            tras_list.append(tras)
            times_rg_list.append(times_rg)

#         # find clusters with unserved customers
#         unserved_cl = []
#         for i in range(self.r):
#             for j in range(self.ts):
#                 if unserves_list[i][j] != [] and idles_list[i][j] == 0:
#                     unserved_cl.append(i)
#         unserved_cl = sorted(unserved_cl,key=unserved_cl.count,reverse=True)
#         list(dict.fromkeys(unserved_cl))
        
#         # find clusters with idle VFNs
#         idle_cl = []
#         for cluster, element in enumerate(idles_list):
#             if min(element) > 0:
#                 idle_cl.append(cluster)
        
#         used = 0
#         vacant = self.distributions_cs[self.time][0]
#         for i in idle_cl:
#             self.w_adjust[i] -= 1
#             vacant += 1
#         for j in list(dict.fromkeys(unserved_cl)):
#             if vacant>used:
#                 self.w_adjust[j] += 1
#                 used += 1

#         # demand adjustement according to service evaluation
#         for i in range(self.r):
#             self.d_adjust[self.packages[self.time][i]][i] = self.w_adjust[i]
#         print("capacity adjustement:", self.d_adjust)

        # calculate profit and servcie rate
        opts_rg_list = np.array(opts_rg_list)
        opts_rg_list[opts_rg_list == None] = 0
        profit = self.opts_cs[self.time]- sum(sum(opts_rg_list))
        print('profit:', profit)
        service_rate = np.nanmean(rates_list)
        print('service rate:', service_rate)
        fulfill_rate = np.nanmean(rates1_list)
        print('fulfill rate:', fulfill_rate)
        return profit, service_rate, fulfill_rate, opts_rg_list, unserves_list, idles_list, rates_list, rates1_list, tras_list, times_rg_list

In [7]:
cp = capacity_planner()
cp.CS_simulation()

Long Time step 0:


C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:146: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:147: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the n

opts_cs: 130.0
cluster 0
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 0.0
Short Time step 2:
RG cost: 0.0
Short Time step 3:
RG cost: 1.01
Short Time step 4:
RG cost: 0.0
Short Time step 5:
RG cost: 0.0
Short Time step 6:
RG cost: 1.01
Short Time step 7:
RG cost: 0.0
Short Time step 8:
RG cost: 0.01
Short Time step 9:
RG cost: 1.01
Short Time step 10:
RG cost: 0.01
Short Time step 11:
RG cost: 9.01
cluster 1
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 0.01
Short Time step 2:
RG cost: 0.0
Short Time step 3:
RG cost: 0.0
Short Time step 4:
RG cost: 0.010000000000000009
Short Time step 5:
RG cost: 0.0
Short Time step 6:
RG cost: 0.0
Short Time step 7:
RG cost: 0.0
Short Time step 8:
RG cost: 0.0
Short Time step 9:
RG cost: 0.010000000000000009
Short Time step 10:
RG cost: 0.0
Short Time step 11:
RG cost: 0.0
cluster 2
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 0.0
Short Time step 2:
RG cost: 0.0
Short Time step 3:
RG cost: 0.0
Short Time ste

Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.
Model has no variables. Nothing to optimize.


RG cost: 1.01
Short Time step 10:
RG cost: 0.010000000000000009
Short Time step 11:
RG cost: 0.01
cluster 4
Short Time step 0:
RG cost: None
Short Time step 1:
RG cost: None
Short Time step 2:
RG cost: None
Short Time step 3:
RG cost: None
Short Time step 4:
RG cost: None
Short Time step 5:
RG cost: None
Short Time step 6:
RG cost: None
Short Time step 7:
RG cost: 0.0
Short Time step 8:
RG cost: 0.0
Short Time step 9:
RG cost: None
Short Time step 10:
RG cost: 0.0
Short Time step 11:
RG cost: None
cluster 5
Short Time step 0:
RG cost: None
Short Time step 1:
RG cost: None
Short Time step 2:
RG cost: None
Short Time step 3:
RG cost: None
Short Time step 4:
RG cost: None
Short Time step 5:
RG cost: None
Short Time step 6:
RG cost: None
Short Time step 7:
RG cost: None
Short Time step 8:
RG cost: 0.0
Short Time step 9:
RG cost: None
Short Time step 10:
RG cost: None
Short Time step 11:


...

C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:146: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:147: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the n

opts_cs: 514.0
cluster 0
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 0.01
Short Time step 2:
RG cost: 1.01
Short Time step 3:
RG cost: 0.0
Short Time step 4:
RG cost: 0.0
Short Time step 5:
RG cost: 0.0
Short Time step 6:
RG cost: 0.01
Short Time step 7:
RG cost: 1.01
Short Time step 8:
RG cost: 0.0
Short Time step 9:
RG cost: 0.0
Short Time step 10:
RG cost: 0.010000000000000009
Short Time step 11:
RG cost: 12.01
cluster 1
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 0.0
Short Time step 2:
RG cost: 0.0
Short Time step 3:
RG cost: 0.010000000000000009
Short Time step 4:
RG cost: 0.0
Short Time step 5:
RG cost: 0.01
Short Time step 6:
RG cost: 0.0
Short Time step 7:
RG cost: 0.01
Short Time step 8:
RG cost: 0.01
Short Time step 9:
RG cost: 0.01
Short Time step 10:
RG cost: 0.0
Short Time step 11:
RG cost: 0.010000000000000009
cluster 2
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 0.0
Short Time step 2:
RG cost: 0.01
Short Time step 3:
RG cos

RG cost: 0.01
profit: 444.12
service rate: 0.9318181818181818
fulfill rate: 0.9715909090909091
Long Time step 23:


C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:146: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:147: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\wenca\Anaconda3\lib\site-packages\ipykernel_launcher.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the n

opts_cs: 914.0
cluster 0
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 1.01
Short Time step 2:
RG cost: 0.010000000000000009
Short Time step 3:
RG cost: 0.010000000000000009
Short Time step 4:
RG cost: 0.0
Short Time step 5:
RG cost: 1.01
Short Time step 6:
RG cost: 0.01
Short Time step 7:
RG cost: 13.01
Short Time step 8:
RG cost: 0.01
Short Time step 9:
RG cost: 0.0
Short Time step 10:
RG cost: 9.01
Short Time step 11:
RG cost: 0.0
cluster 1
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 0.01
Short Time step 2:
RG cost: 0.01
Short Time step 3:
RG cost: 0.01
Short Time step 4:
RG cost: 0.01
Short Time step 5:
RG cost: 0.0
Short Time step 6:
RG cost: 0.0
Short Time step 7:
RG cost: 0.01
Short Time step 8:
RG cost: 0.010000000000000009
Short Time step 9:
RG cost: 0.0
Short Time step 10:
RG cost: 0.01
Short Time step 11:
RG cost: 0.01
cluster 2
Short Time step 0:
RG cost: 0.0
Short Time step 1:
RG cost: 0.0
Short Time step 2:
RG cost: 0.0
Short Time step 3:
RG 

In [8]:
cp.profit_list

[107.57,
 121.51,
 134.03,
 165.94,
 196.41,
 1170.6399999999999,
 3163.17,
 3972.09,
 2497.3500000000004,
 3728.44,
 3752.3999999999996,
 1630.98,
 2019.22,
 3472.57,
 3427.99,
 3898.46,
 3604.59,
 1774.81,
 346.2,
 187.41,
 402.65999999999997,
 388.65999999999997,
 444.12,
 793.81]

In [9]:
sum(cp.profit_list)

41401.030000000006

In [10]:
sum(sum(sum(cp.opts_rg_block)))

9544.970000000001

In [11]:
cp.service_rate_list

[0.9147286821705425,
 0.9727891156462586,
 0.9432624113475176,
 0.9449404761904763,
 0.9265536723163842,
 0.9217582417582417,
 0.895179521792425,
 0.8850548552754435,
 0.727802522837189,
 0.8077466594707973,
 0.9016880561748508,
 0.9286056868790684,
 0.9444444444444443,
 0.9135416666666667,
 0.9101094796407296,
 0.897010420850627,
 0.9141025180617941,
 0.9432614160033513,
 0.9447916666666668,
 0.9505649717514123,
 0.9375,
 0.9323529411764706,
 0.9318181818181818,
 0.9305322128851541]

In [12]:
sum(cp.service_rate_list)/len(cp.service_rate_list)

0.9133391592426957

In [13]:
cp.fulfill_rate_list

[0.9651162790697675,
 1.0,
 0.9787234042553191,
 0.9761904761904762,
 0.9915254237288136,
 0.957051282051282,
 0.9098566308243728,
 0.8881866635543105,
 0.7347349349891539,
 0.8138712956446946,
 0.9033770651117591,
 0.939388489208633,
 0.9497979797979799,
 0.9197312801932367,
 0.9251837572150072,
 0.9065967108492883,
 0.9198412698412698,
 0.9498387096774195,
 0.971875,
 0.9830508474576272,
 0.9702380952380952,
 0.9519607843137255,
 0.9715909090909091,
 0.9628851540616246]

In [14]:
sum(cp.fulfill_rate_list)/len(cp.fulfill_rate_list)

0.9350255184318653

In [15]:
[cp.n_vfn - cp.distributions_cs[i][0] for i in range(24)]

[17.0,
 19.0,
 16.0,
 19.0,
 19.0,
 22.0,
 30.0,
 30.0,
 30.0,
 30.0,
 30.0,
 24.0,
 22.0,
 30.0,
 30.0,
 30.0,
 30.0,
 24.0,
 20.0,
 20.0,
 20.0,
 20.0,
 20.0,
 20.0]

In [16]:
cp.times_cs

[0.1452648639678955,
 0.04944610595703125,
 0.07126903533935547,
 0.0630028247833252,
 0.06560993194580078,
 0.08503913879394531,
 0.11794042587280273,
 0.13229751586914062,
 0.14120078086853027,
 0.14122438430786133,
 0.14103460311889648,
 0.10982108116149902,
 0.10983848571777344,
 0.13596010208129883,
 0.1879267692565918,
 0.1380157470703125,
 0.14113688468933105,
 0.11023831367492676,
 0.09418082237243652,
 0.0941925048828125,
 0.09423995018005371,
 0.07854175567626953,
 0.10991907119750977,
 0.09420442581176758]

In [17]:
[np.mean(np.array(i)) for i in cp.times_rg_block]

[0.011199580629666646,
 0.013062403599421183,
 0.011541047692298889,
 0.012476003170013428,
 0.013162978490193685,
 0.022036167979240417,
 0.052748066186904904,
 1.002493562301,
 6.581342631578446,
 0.09501901169617971,
 0.09147822360197704,
 0.03154042363166809,
 0.03262574076652527,
 0.39602171381314594,
 0.07405360738436381,
 0.07031948864459991,
 0.07613099813461303,
 0.03483512004216512,
 0.016655450065930683,
 0.015029304226239522,
 0.016846923033396403,
 0.01707639396190643,
 0.01693209707736969,
 0.019757906595865887]

In [18]:
[np.max(np.array(i)) for i in cp.times_rg_block]

[0.03235483169555664,
 0.06553173065185547,
 0.04729652404785156,
 0.04576539993286133,
 0.047272682189941406,
 0.18641138076782227,
 0.8459210395812988,
 187.72439789772034,
 300.42711544036865,
 1.6944701671600342,
 4.378672122955322,
 0.21973729133605957,
 0.27617502212524414,
 76.54717874526978,
 1.6165058612823486,
 0.5808169841766357,
 1.741894245147705,
 0.34515857696533203,
 0.0629875659942627,
 0.09420609474182129,
 0.06250262260437012,
 0.1032247543334961,
 0.07854461669921875,
 0.14143133163452148]